## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,28.42,86,90,19.57,mist
1,1,Umluj,SA,25.0213,37.2685,85.05,54,97,4.99,overcast clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.71,78,4,21.16,clear sky
3,3,Iberia,US,29.9502,-91.7507,76.96,43,1,6.91,clear sky
4,4,Beringovskiy,RU,63.0500,179.3167,6.53,92,97,12.59,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [9]:
# 3. Filter the DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
14,14,Fortuna,US,40.5982,-124.1573,66.92,66,40,7.00,scattered clouds
15,15,Vila Velha,BR,-20.3297,-40.2925,71.55,94,90,10.36,light rain
17,17,Mahébourg,MU,-20.4081,57.7000,72.25,83,75,3.44,broken clouds
27,27,Beloha,MG,-25.1667,45.0500,73.33,74,98,4.61,light rain
28,28,Abu Samrah,SY,35.3029,37.1841,69.22,51,98,15.75,overcast clouds
31,31,Lompoc,US,34.6391,-120.4579,71.69,62,1,5.75,clear sky
34,34,Port Alfred,ZA,-33.5906,26.8910,67.12,83,100,17.02,overcast clouds
49,49,Rikitea,PF,-23.1203,-134.9692,73.80,87,78,6.40,light rain
56,56,Chui,UY,-33.6971,-53.4616,71.49,78,46,9.93,scattered clouds
57,57,Saint-François,FR,46.4154,3.9054,65.64,59,90,11.50,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Fortuna,US,66.92,scattered clouds,40.5982,-124.1573,
15,Vila Velha,BR,71.55,light rain,-20.3297,-40.2925,
17,Mahébourg,MU,72.25,broken clouds,-20.4081,57.7000,
27,Beloha,MG,73.33,light rain,-25.1667,45.0500,
28,Abu Samrah,SY,69.22,overcast clouds,35.3029,37.1841,
31,Lompoc,US,71.69,clear sky,34.6391,-120.4579,
34,Port Alfred,ZA,67.12,overcast clouds,-33.5906,26.8910,
49,Rikitea,PF,73.80,light rain,-23.1203,-134.9692,
56,Chui,UY,71.49,scattered clouds,-33.6971,-53.4616,
57,Saint-François,FR,65.64,overcast clouds,46.4154,3.9054,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df.loc[index, "Lat"] 
    lng = hotel_df.loc[index, "Lng"] 

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params)
    results = response.json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel = results['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel
    except (IndexError):
        print(f"Hotel in {hotel_df.loc[index, 'City']} not found, skipping city")

Hotel in Beloha not found, skipping city
Hotel in Abu Samrah not found, skipping city
Hotel in Orange Cove not found, skipping city
Hotel in Lodja not found, skipping city
Hotel in Durango not found, skipping city
Hotel in Rafai not found, skipping city
Hotel in Gambo not found, skipping city
Hotel in Jinji not found, skipping city


In [50]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Fortuna,US,66.92,scattered clouds,40.5982,-124.1573,Super 8 by Wyndham Fortuna
15,Vila Velha,BR,71.55,light rain,-20.3297,-40.2925,Quality Suites Vila Velha
17,Mahébourg,MU,72.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
27,Beloha,MG,73.33,light rain,-25.1667,45.0500,
28,Abu Samrah,SY,69.22,overcast clouds,35.3029,37.1841,
...,...,...,...,...,...,...,...
687,Fālākāta,IN,66.58,clear sky,26.5208,89.2055,Hotel Nandanik Palace
692,Abong Mbang,CM,68.99,light rain,3.9833,13.1833,Auberge Angle Sur
694,Tomohon,ID,65.34,overcast clouds,1.3346,124.8392,Gardenia Country Inn
703,Gwanda,ZW,73.47,clear sky,-20.9333,29.0000,Mafax Homeways


In [66]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Fortuna,US,66.92,scattered clouds,40.5982,-124.1573,Super 8 by Wyndham Fortuna
15,Vila Velha,BR,71.55,light rain,-20.3297,-40.2925,Quality Suites Vila Velha
17,Mahébourg,MU,72.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
31,Lompoc,US,71.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
34,Port Alfred,ZA,67.12,overcast clouds,-33.5906,26.8910,The Halyards Hotel
49,Rikitea,PF,73.80,light rain,-23.1203,-134.9692,People ThankYou
56,Chui,UY,71.49,scattered clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
57,Saint-François,FR,65.64,overcast clouds,46.4154,3.9054,Manoir Du Bonheur
59,Kruisfontein,ZA,69.89,broken clouds,-34.0033,24.7314,Oyster Bay House Rental
64,Ajdabiya,LY,69.89,scattered clouds,30.7554,20.2263,Almadina Hotel


In [67]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [73]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))